# 사용한 라이브러리

 - pandas
 - numpy
 - matplotlib
 - os
 - sklearn
 - tensorflow(keras) => pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Bidirectional
from keras.utils import to_categorical, np_utils
from keras import backend as K

# 전처리 과정
---

In [ ]:
# 서울인지 경기인지 따로 path에 작성
path = '../data/(여기에 서울이면 seoul 경기도면 gyeonggi 작성)/'
# 촐퇴근 시간 포함할지 안할지 True가 포함 X
commute_time = True

main_data = pd.DataFrame()

for i in os.listdir(path):
    #경기도 버스 csv 파일을 거르기 위한 if문
    if 'G' not in i:
        #종종 csv 파일에 공백이 있는경우가 있는데 그러면 오류 나와서 이를 해결하기위해 만듦 
        try:
            temp_data = pd.read_csv(path + i, names=['계절', '시간', '버스', '정류장', '혼잡도',
                                                     '평휴일', '이전휴일수', '이후휴일수'])
            main_data = pd.concat([main_data, temp_data], axis = 0, ignore_index = True)
        except:
            continue
            
# 혹시나 혼잡도가 3~6사이가 아닐경우 삭제
idx = main_data[((main_data['혼잡도'] < 3) & (main_data['혼잡도'] > 6))].index
print(idx)
main_data.drop(idx, inplace=True)

# 출퇴근 시간 확인
if commute_time:
            idx = main_data[((main_data['시간'] >= 700) & (main_data['시간'] <= 1000)) |
                            ((main_data['시간'] >= 1630) & (main_data['시간'] <= 2000))].index
            main_data.drop(idx, inplace=True)
            
main_data = main_data.astype('float32')

# 버스와 시간은 스케일링 했습니다. (0~1사이로 조정)
main_data['시간'] = main_data['시간'] / 2400
main_data['버스'] = main_data['버스'] / 682

y = main_data['혼잡도']
del main_data['혼잡도']

y = y-2
y = to_categorical(y)

#numpy array형식으로 변경 후 LSTM 모델 형식에 맞게 reshape
X = np.array(main_data)
X = X.reshape(len(X), len(X[0]), 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=777)

# 모델 설계

 - LSTM 모델로 설정
 - 길이는 우리 파일 길이에 맞게 설정해놓았습니다.
 - 따로 건드릴건 없습니다!
---

In [ ]:
K.clear_session()
xInput = Input(batch_shape=(None,X_train.shape[1], X_train.shape[2]))
xLstm_1 = LSTM(len(X_train[0]), return_sequences= True)(xInput)
xLstm_2 = Bidirectional(LSTM(10))(xLstm_1)
xOutput = Dense(4, activation= 'softmax')(xLstm_2)
model = Model(xInput, xOutput)
model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

# 모델 학습

 - 모델 학습 단계입니다.
 - epochs는 학습 횟수입니다.(한번 돌려보시고 데이터가 많아서 느리다면 에폭을 줄여주세요.)
 - loss가 1이하로 나온다면 많이 안하셔도 됩니다.
 - loss가 내려가지않고 유지된다면 이제 더이상 학습하기 힘들수도 있습니다.
 - batch_size는 한번에 얼마나 많은 데이터를 학습시킬지 입니다. 데이터가 부족하면 줄이고 많다면 늘려주세요.
 - verbose는 신경안써도 됩니다!
---

In [ ]:
model.fit(X_train, y_train, epochs=2, batch_size=20)

# 모델 테스트

 - 모델 테스트 단계입니다.
 - test 데이터가 100000개가 넘어가면 자르게 설정했습니다.
 - X_backup과 y_backup에 따로 백업하고 자릅니다!
---

In [ ]:
if len(X_test) > 100000:
    X_backup = X_test
    y_backup = y_test
    X_test = X_test[0:100000]
    y_test = y_test[0:100000]
y_hat =(model.predict(X_test) > 0.5).astype("int32")

# 결과 확인

 - 결과 확인 입니다!
 - 첫번째는 부분 확인입니다.
  - start_line에 원하는 숫자과 end_line에 start_line보다 큰수를 입력하면 결과 확인할 수 있습니다.
 
---

In [ ]:
start_line = 0
end_line = 200

result = []
label = []
for i in y_hat:
    result.append(i.tolist().index(1))
for i in y_test:
    label.append(i.tolist().index(1))
b_axis = np.arange(0, len(result[start_line:end_line]))

plt.figure(figsize=(20,8))
plt.plot(b_axis, result[start_line:end_line], 'o-', color='red', label='Predicted')
plt.plot(b_axis, label[start_line:end_line], 'o-', color='green', alpha=0.2, label='Actual')
plt.legend()
plt.show()

In [ ]:
result = []
label = []
for i in y_hat:
    result.append(i.tolist().index(1))
for i in y_test:
    label.append(i.tolist().index(1))
b_axis = np.arange(0, len(result))

plt.figure(figsize=(20,8))
plt.plot(b_axis, result, 'o-', color='red', label='Predicted')
plt.plot(b_axis, label, 'o-', color='green', alpha=0.2, label='Actual')
plt.legend()
plt.show()